In [14]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

import math

Все задания выполняются на основе датасета UCI ML Breast Cancer Wisconsin. (https://goo.gl/U2Uwz2)  
Все признаки являются числовыми.

In [15]:
X, y = load_breast_cancer(return_X_y=True)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)


1) Реализовать функцию подсчета индекса Джини

In [17]:
def compute_gini_impurity(labels):
    """
    :arg labels: np.array of shape (n_objects,)
    
    :return: gini_impurity: float
    """
    unique, counts = np.unique(labels, return_counts=True)

    rez = 1
    for i in range(len(unique)):
        veroatn =  counts[i]/np.sum(counts)
        rez = rez - veroatn ** 2

    return rez

2) Реализовать функцию подсчета энтропии

In [18]:
def compute_entropy(labels):
    """
    :arg labels: np.array of shape (n_objects,)
    
    :return: entropy: float
    """
    unique, counts = np.unique(labels, return_counts=True)

    rez = 0
    for i in range(len(unique)):
        veroatn =  counts[i]/np.sum(counts)
        rez = rez - veroatn*math.log2(veroatn)

    return rez

3) Реализовать функцию подсчета Information Gain

In [19]:
def calculate_information_gain(s0, *args):
    """
    Calculates IG for a split s_1, ..., s_k.
        :arg s0: the whole set, sequence of shape (n_objects,)
        :arg args: split — a list of sequences, such that s0 = union(s_1, ..., s_k) and
           for all i, j: intersection(s_i, s_j) = 0
           
        :return: information_gain: float
    """
    unique_s0, counts_s0 = np.unique(s0, return_counts=True)

    unique_args = []
    counts_args = []

    for i in range(len(args)):
        unique_buf, counts_buf = np.unique(args[i], return_counts=True)
        unique_args.append(unique_buf)
        counts_args.append(counts_buf)

    rez = compute_entropy(s0)

    for i in range(len(args)):
        rez = rez - (np.sum(counts_args[i])/np.sum(counts_s0))*compute_entropy(args[i])

    return rez

In [20]:
calculate_information_gain([1, 2, 3, 4, 5, 6], [1, 2, 3, 4], [5, 6])

0.9182958340544896

В следующих заданих требуется так или иначе изменить параметры дерева решений и оценить результат.  
Как метрику качества будем использовать F1-score. В каждом задании необходимо поменять __только один__ параметр.

In [21]:
def compute_metric(clf):
    np.random.seed(42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred), 6)

Посчитаем значение метрики со стандартными параметрами.

In [22]:
compute_metric(DecisionTreeClassifier())

0.957746

4) Добавьте следующее условие: для выполнения сплита должно быть как минимум 22 объекта. Чему равна метрика F1 на отложенной выборке?

In [23]:
compute_metric(DecisionTreeClassifier(min_samples_split=22))

0.951049

5) Добавьте следующее условие: в листе должно быть как минимум 10 элементов. Чему равна метрика F1 на отложенной выборке?

In [27]:
compute_metric(DecisionTreeClassifier(max_leaf_nodes=10))

0.957746

6) Добавьте следующее условие: в дереве должно быть не более 8 листьев. Чему равна метрика F1 на отложенной выборке?

In [28]:
compute_metric(DecisionTreeClassifier(min_samples_leaf=8))

0.965035

7) Добавьте следующее условие: глубина дерева должна быть не более 4. Чему равна метрика F1 на отложенной выборке?

In [26]:
compute_metric(DecisionTreeClassifier(max_depth=4))

0.957746

8) При помощи параметра "class_weight" выставите веса объектов обратно пропорциональными их частоте в обучающей выборке. Чему стала равна метрика F1 на отложенной выборке?

In [16]:
compute_metric(DecisionTreeClassifier(class_weight="balanced"))

0.965035

9) Поменяйте критерий для построения дерева решений на 'entropy', который мы разбирали на теории. Чему стала равна метрика F1 на отложенной выборке?

In [17]:
compute_metric(DecisionTreeClassifier(criterion="entropy"))

0.958904

10) Укажите параметр min_samples_split=50. Какая получилась глубина у получившегося дерева? Укажите ее ниже.

In [18]:
compute_metric(DecisionTreeClassifier(min_samples_split=50))

0.937931

11) Вставьте в код условие, которое выполнит эвристику `min_samples_split`.

```python
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
        self.root = None
    
    def _fit_id3(self, x, y, level):
        class_labels = np.unique(y)
        # if there is only one class in y then return leaf node
        if class_labels.size == 1:
            self.n_nodes += 1
            return Node(class_label=class_labels[0])

        # find the most informative predicate and split by them
        beta = Predicate(x, y)
        to_left, to_right = beta.split(x, y)

        # if one of arrays is empty after split then return leaf node
        if (to_left[0].size == 0 or 
            to_right[0].size == 0 or 
            level > self.max_depth or
            _______________________):
            labels, counts = np.unique(y, return_counts=True)
            class_label = labels[counts.argmax(axis=0)]
            self.n_nodes += 1
            return Node(class_label=class_label)

        node = Node()
        node.set_predicate(beta)

        node.set_left(self._fit_id3(to_left[0], to_left[1], level + 1))
        node.set_right(self._fit_id3(to_right[0], to_right[1], level + 1))

        return node
```

In [19]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
        self.root = None
    
    def _fit_id3(self, x, y, level):
        class_labels = np.unique(y)
        # if there is only one class in y then return leaf node
        if class_labels.size == 1:
            self.n_nodes += 1
            return Node(class_label=class_labels[0])

        # find the most informative predicate and split by them
        beta = Predicate(x, y)
        to_left, to_right = beta.split(x, y)

        # if one of arrays is empty after split then return leaf node
        if (to_left[0].size == 0 or to_right[0].size == 0 or level > self.max_depth or self.min_samples_split >= 2):
            labels, counts = np.unique(y, return_counts=True)
            class_label = labels[counts.argmax(axis=0)]
            self.n_nodes += 1
            return Node(class_label=class_label)

        node = Node()
        node.set_predicate(beta)

        node.set_left(self._fit_id3(to_left[0], to_left[1], level + 1))
        node.set_right(self._fit_id3(to_right[0], to_right[1], level + 1))

        return node

12) Вставьте в код условие, которое выполнит эвристику `min_samples_leaf`.

```python
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
        self.root = None
    
    def _fit_id3(self, x, y, level):
        class_labels = np.unique(y)
        # if there is only one class in y then return leaf node
        if class_labels.size == 1 or ________________:
            self.n_nodes += 1
            return Node(class_label=class_labels[0])

        # find the most informative predicate and split by them
        beta = Predicate(x, y)
        to_left, to_right = beta.split(x, y)

        # if one of arrays is empty after split then return leaf node
        if (to_left[0].size == 0 or 
            to_right[0].size == 0 or 
            level > self.max_depth):
            labels, counts = np.unique(y, return_counts=True)
            class_label = labels[counts.argmax(axis=0)]
            self.n_nodes += 1
            return Node(class_label=class_label)

        node = Node()
        node.set_predicate(beta)

        node.set_left(self._fit_id3(to_left[0], to_left[1], level + 1))
        node.set_right(self._fit_id3(to_right[0], to_right[1], level + 1))

        return node
```

In [20]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        
        self.root = None
    
    def _fit_id3(self, x, y, level):
        class_labels = np.unique(y)
        # if there is only one class in y then return leaf node
        if class_labels.size == 1 or min_samples_leaf >= 1:
            self.n_nodes += 1
            return Node(class_label=class_labels[0])

        # find the most informative predicate and split by them
        beta = Predicate(x, y)
        to_left, to_right = beta.split(x, y)

        # if one of arrays is empty after split then return leaf node
        if (to_left[0].size == 0 or to_right[0].size == 0 or level > self.max_depth):
            labels, counts = np.unique(y, return_counts=True)
            class_label = labels[counts.argmax(axis=0)]
            self.n_nodes += 1
            return Node(class_label=class_label)

        node = Node()
        node.set_predicate(beta)

        node.set_left(self._fit_id3(to_left[0], to_left[1], level + 1))
        node.set_right(self._fit_id3(to_right[0], to_right[1], level + 1))

        return node